The fairness of the ranking system of the [LLM 20 Questions competition](https://www.kaggle.com/competitions/llm-20-questions/overview) has been questioned multiple times in discussions, so I decided to write a small toying notebook to evaluate it.  We're not going to evaluate the maths, just play with data.

# Official description of the ranking system

At the time of writing (July 17th), the competition's overview states :
> ### Evaluation
>
> Each day your team is able to submit up to 5 agents (bots) to the competition. Each submission will play episodes (games) against other bots on the leaderboard that have a similar skill rating. Over time, skill ratings will go up with wins, down with losses, or evened out with ties.
> 
> This competition is configured to run in a cooperative, 2 vs. 2 format. Your bot will be randomly paired with a bot of similar skill in order to face off against another random pairing. On each pair, one bot will be randomly assigned as questioner and the other as answerer. Since you win/lose/tie as a pair, you are incentivized to work together!
> 
> Every bot submitted will continue to play episodes until the end of the competition, with newer bots selected to play more frequently. Once you have reached three active submissions, older ones will be deactivated. On the leaderboard, only your best scoring bot will be shown, but you can track the progress of all of your submissions on your Submissions page.
> 
> Each submission has an estimated skill rating which is modeled by a Gaussian N(μ,σ2) where μ is the estimated skill and σ represents the uncertainty of that estimate which will decrease over time.
> 
> When you upload a submission, we first play a validation episode where that submission plays against copies of itself to make sure it works properly. If the episode fails, the submission is marked as error and you can download the agent logs to help figure out why. Otherwise, we initialize the submission with μ0=600 and it joins the pool of for ongoing evaluation. At this time we also deactivate older agents if the total number of active agents is greater than three.
> 
> ### Ranking System
> 
> After an episode finishes, we'll update the rating estimate for all bots in the episode. If one bot pair won, we'll increase their μ and decrease the opponent's μ -- if the result was a tie, then we'll move the μ values closer towards their mean. The updates will have magnitude relative to the deviation from the expected result based on the previous μ values, and also relative to each bot’s uncertainty σ. We also reduce the σ terms relative to the amount of information gained by the result. The score by which your bot wins or loses an episode does not affect the skill rating updates.
> 
> ### Final Evaluation
> 
> At the submission deadline on August 13, 2024, submissions will be locked. From August 13, 2024 to August 27th, 2024 we will continue to run episodes against a new set of unpublished, secret words. During this period only your three active submissions will be eligible for the leaderboard. At the conclusion of this period, the leaderboard is final.

# TrueSkill

I don't know much about rating systems.  A quick search shows that a candidate for a system with a $\mu$ **and** $\sigma$ parameters may be the [Glicko rating system](https://en.wikipedia.org/wiki/Glicko_rating_system).  I can't remember how I came to this conclusion (maybe it had to do with team play ?), but another more likely candidate is [TrueSkill](https://en.wikipedia.org/wiki/TrueSkill).  I won't go into details, follow the Wikipedia link if you're interested.  It seems quite popular in the world of video games.  There is a [TrueSkill 2](https://www.microsoft.com/en-us/research/uploads/prod/2018/03/trueskill2.pdf), but at first glance its improvements don't seem relevant for our kind of competition.

## Installation

Fortunately, there is a [Python library](https://github.com/sublee/trueskill) of TrueSkill available on [Pypi](https://pypi.org/project/trueskill/), let's install it :

In [ ]:
!pip install trueskill

Players ratings are represented by a `Rating` class, and a game is played by `rate()`.

In [ ]:
from trueskill import Rating, rate

## Let's play

Let's define four players with $\mu=600$ :

In [ ]:
player1 = Rating(mu=600)
player2 = Rating(mu=600)
player3 = Rating(mu=600)
player4 = Rating(mu=600)

teamA = [ player1, player2 ]
teamB = [ player3, player4 ]

Now let them play a game :

In [ ]:
rate(
    [ teamA, teamB ],
    [ 0, 1 ]  # teamA is ranked lower than teamB, so teamA won
)

The `rate()` function returns the new ratings for each players, by teams.

## Simple display function

It will be convenient to define a function to help reading the consequences of a game :

In [ ]:
from typing import Tuple

def play_game(
    teamA: Tuple[Rating, Rating],
    teamB: Tuple[Rating, Rating],
    outcome: Tuple[int, int]
):
    """
    Play a game between two teams and display the effect on the players' ratings.
    
    Params
    ------
        teamA: Tuple[Rating, Rating]
            A team of two players.
        teamB: Tuple[Rating, Rating]
            A team of two players.
        outcome: Tuple[int, int]
            The ranking of the players in the game (0: first, 1: second)
    """
    new_ratings = rate(
        [ teamA, teamB ],
        outcome
    )
    for old,new in zip(teamA, new_ratings[0]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
    print("vs")
    for old,new in zip(teamB, new_ratings[1]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")

Applying it on our previous players :

In [ ]:
play_game(teamA, teamB, [0, 1])

## Finding sigma

Now it will quick become obvious that the competition doesn't use the $\sigma$ default value.  The TrueSkill recommandation is to set $\sigma = \frac{\mu}{3}$, but after playing a bit with these parameters I believe the relation for this competition has rather been set to $\sigma = \frac{\mu}{2} = 300$.

Indeed, let's try to reproduce the outcome of an actual episode.  Here is the first win of my first submission :

> [1st] gguillard 600 (+123) vs [1st] JavaZero 570 (+34) vs
>
> [3rd] atom1231 577 (-72) vs [3rd] KKY 464 (-44)

With a little trial and error, and assuming my $\sigma$ would not be very far than its initialization at 300, it is quick and easy to find a matching combination of parameters :

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 0, 1 ]  # first team won
)

## The draw probability

So is the scoring system TrueSkill with $\mu=600$ and $\sigma=300$ ?  Well, not quite.  These settings don't work well in case of draws, the variations are much larger than what can be observed in the competition (typically 0 to ± a few units) :

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 1, 1 ]  # no winner
)

Reading again the rules, the keypoint is here :

> We also reduce the σ terms relative to the amount of information gained by the result.

At first I played a bit with parameters and thought $\sigma$ was manually reduced by a factor 100 for draws, it seemed to work fairly well.  Then reading `help(trueskill)` I figured it may already be taken into account within TrueSkill, since the V and W functions, responsible for the $\mu$ and $\sigma$ update, respectively, have a "win" **and** a "draw" version.

Actually, one can define a TrueSkill environment with a few more parameters, including `draw_probability`.  That's interesting, because we all know the draw probability for a game in this competition is… *fairly high*.

Let's compute it, we'll need it for our simulation.  We'll use @waechter's [LLM 20 Questions - Games dataset](https://www.kaggle.com/code/waechter/llm-20-questions-games-dataset).

In [ ]:
import pandas as pd
from ast import literal_eval

games_df = pd.read_csv(
    "/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv",
    converters = {
        col_list: literal_eval
        for col_list in ["answers", "questions", "guesses"]
    }
)
games_df = games_df[games_df["guesser_SubmissionId"]!=0]  # it seems that submissions of the day are not yet labelled
games_df["CreateTime"] = pd.to_datetime(games_df["CreateTime"], format="%m/%d/%Y %H:%M:%S")

This dataframe contains two rows per game, one for each team, and a boolean "guessed" column.  In case of a draw, "guessed" is the same for both teams (although we could safely ignore the 3 games where both teams won at the same round…).

In [ ]:
games_df.groupby("game_num")["guessed"].sum().value_counts()

In [ ]:
(144607 + 3) / (144607 + 1208 + 3)

Therefore, the probability of a draw is : $p \simeq 0.9917$.  Fairly high indeed…  We'll update it later with more accurate data (spoiler : it'll only get worse).

## Beta and tau

The documentation also mentions two other parameters that may be of interest :

>  :param beta: the distance which guarantees about 76% chance of winning.
>               The recommended value is a half of ``sigma``.
>
>  :param tau: the dynamic factor which restrains a fixation of rating.  The
>              recommended value is ``sigma`` per cent.

Let's use the recommended values, namely $\beta = \frac{\sigma}{2} = 150$ and $\tau = \frac{\sigma}{100} = 3$.

Now let's define our new TrueSkill environment :

In [ ]:
from trueskill import setup

setup(
    mu = 600,
    sigma = 300,
    beta = 150,
    tau = 3,
    draw_probability = 0.9917
)

It takes a little bit of adjustment again to get the right $\sigma$ :

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 0, 1 ]  # first team won
)

But then, ratings update after a draw are much more consistent with what we actually observe in the competition :

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 1, 1 ]  # no winner
)

It looks to me that we managed to construct a scoring system quite similar to the one used by the LLM 20 Questions competition.  It is possible that the parameters are not *exactly* those ones, and anyway the draw probability couldn't have been deduced before a certain number of games (maybe it's updated continuously ? – it also accepts a function, BTW), but I would be surprised if the ability of these settings to reproduce the results of a game was a coincidence.

# Modeling the competition

## Winning probabilities

Now what about generating a full board of players, teaming them up randomly and having them play ? :D

Let's first extract some realistic probabilities of winning.  Using the *LLM 20 Questions - Games dataset* again, we'll compute the `nb_guessed / nb_played` ratio for each submission.

But in order to get a realistic distribution of skills, we will first filter out submissions with a small number of games played (say 10) :

In [ ]:
nb_games_played = games_df.groupby("guesser_SubmissionId").size()
nb_games_played = nb_games_played[nb_games_played>=10]

In [ ]:
games_df = games_df[games_df["guesser_SubmissionId"].isin(nb_games_played.index)]

Since the competition only allows three submissions per competitor, we also have to remove all bots but the last three — some people have more than 100 submissions !

In [ ]:
games_df.groupby("guesser")["guesser_SubmissionId"].nunique().sort_values(ascending=False).head()

In [ ]:
last_bots = games_df.sort_values(
    by = "CreateTime"  # don't sort by "guesser" here because some submissions have several names
).drop_duplicates(
    subset = "guesser_SubmissionId",
    keep = "last"
).groupby(
    [ "guesser" ]
)[[
    "guesser",
    "guesser_SubmissionId",
    "CreateTime"
]].tail(3).sort_values(
    by = "guesser"
)["guesser_SubmissionId"].values

In [ ]:
last_games_df = games_df[games_df["guesser_SubmissionId"].isin(last_bots)]

Time to update our draw probability :

In [ ]:
last_games_df.groupby("game_num")["guessed"].sum().value_counts()

In [ ]:
(123874 + 1) / (123874 + 546 + 1)

As mentioned earlier, it's even worse than before, with the probability of a draw being $p \simeq 0.9956$…

We'll need to distinguish between "good bots" and "dummy bots" to compute the actual skills distribution.

In [ ]:
goodbots = set(
    last_games_df[
        last_games_df.groupby("guesser_SubmissionId")["guessed"].transform("any")
    ]["guesser_SubmissionId"].unique()
)

In [ ]:
badbots = set(last_games_df["guesser_SubmissionId"].unique()) - goodbots

In [ ]:
len(goodbots), len(badbots)

Now let's consider every team of good bots.  We don't care about the opponents being dumb, because it only takes a reliable teammate to find the keyword.  We will also discard sessions in which the team didn't have time to find the keyword because the other team did before, as this would only bias our simulation.  On the other hand, we keep instances in which the keyword was guessed although the teammate is in the badbots category, because it may just be that the teammate didn't have a chance to win yet.  Incidentally, removing these cases would drastically reduce the statistics.

In [ ]:
fair_games = last_games_df["guesser_SubmissionId"].isin(goodbots) * (
    last_games_df["answerer_SubmissionId"].isin(goodbots) + last_games_df["guessed"]
)
fair_games_df = last_games_df.loc[fair_games]
fair_games_df = fair_games_df[(fair_games_df["guessed"]) | (fair_games_df["nb_round"]==20)]

In [ ]:
skills_df = fair_games_df.groupby("guesser_SubmissionId").agg(
    guessed = ("guessed", "sum"),
    played = ("guessed", "size")
).sort_index()
skills_df["skill"] = skills_df["guessed"] / skills_df["played"]
skills_df.sort_values(by = "skill", ascending = False, inplace = True)
skills_df.head()

Let's remove bots with less than $N = 10$ fair games in order to avoid unrealistic probabilities (we already did it for *all* games, but *fair* games is an other story) :

In [ ]:
skills_df = skills_df[skills_df["played"]>=10]
skills_df.head()

Finally, let's increase the dynamic range of skills by allowing the best guesser to be a superguesser with 98 % of chances to find the keyword.  Thats very optimistic !  But this will also be representative of the evolution towards better bots as we approach the end of the competition.

In [ ]:
BEST_SKILL = .98
skills_df["skill"] *= BEST_SKILL / skills_df["skill"].iloc[0]

Now we have a somehow realistic representation of the global skill distribution of the current (legit) bots.  Let's have a look at it :

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,6))
_ = plt.hist(skills_df["skill"], bins=15)
_ = plt.xlabel("Probability to find the keyword against a fair teammate")
_ = plt.ylabel("Number of bots")
_ = plt.title("Good bots skill distribution")

That should lead to a clear leaderboard !

## A Player class

Now let's create a Player class in order to keep track of the ratings history :

In [ ]:
from typing import Optional
from trueskill import TrueSkill

class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # ignore this for now, this will be useful later
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # start with environment default values, i.e. (600,300)
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating

## A pool of players

We can then build a pool of players, some with a range of realistic skills, along with a number of dummy bots.  Since there was a significant reduction of the number of good bots considered for the skills evaluation, it seems fair to apply the same reduction ratio to the number of bad bots as well, but anyway it shouldn't change much the outcome of the simulation (actually it may get worse…).  Play with it if you wish.

In [ ]:
reduction_ratio = len(skills_df) / len(goodbots)

players = { ii: Player(s) for ii,s in enumerate(skills_df["skill"].values) }
players.update({ len(players)+ii: Player(0) for ii in range(int(len(badbots)*reduction_ratio)) })

len(players)

We still have more players than in the actual leaderboard, but that's the number of bots, remember each competitor on the leaderboard may have up to three active submissions.

## A game function

Finally, we need to define a game function taking into account the winning probability.

In [ ]:
import random

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill] = None,
    debug: bool = False
):
    """
    Plays a match between two teams.
    
    Params
    ------
        teamA: Tuple[Player, Player]
            A team of two players.
        teamB: Tuple[Player, Player]
            Another team of two players.
        debug: boolean
            A flag for debugging.
    """
    if debug:
        print("Players skills :", [p.skill for p in teamA+teamB])
        print("Players ratings :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # for each team, if any player is a dummy bot, the team fails to find the keyword
    # else the team finds the keyword if its guesser's skill is higher than a random number
    # (beware of the reversed logic in the code)
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # new ratings are the results of the game
    rating_function = rate if env is None else env.rate
    new_ratings = rating_function(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # update all players' ratings
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("Game outcome :", ranks)
        print("New ratings :", new_ratings)
    
    return

And voilà !  We have everything we need to start simulating the competition.  Here is a first game :

In [ ]:
play_match(
    [players[0], players[1]],
    [players[2], players[3]],
    debug = True
)

# Make your own competition simulation

For convenience and the ability to test different settings easily, we will define a competition class where we can parametrize the number and skills of players, the number of games and the TrueSkill settings, and which provides a dataframe with the ratings history.

In [ ]:
from typing import List
from tqdm import tqdm

In [ ]:
class Competition:
    """
    A configurable TrueSkill competition environment.
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42
    ):
        """
        Constructor.
        
        Params
        ------
            skill_dist: List[float]
                The probability to find the keyword, for each "good" submission.
            nb_dummy: int
                The number of dummy bots
            mu: float
                The TrueSkill mu parameter.
            sigma: float
                The TrueSkill sigma parameter.
            beta: float
                The TrueSkill beta parameter.
            tau: float
                The TrueSkill tau parameter.
            draw_probability: float
                The TrueSkill draw_probability parameter.
            seed: float
                A random seed.
        """
        # our competition environment
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # our pool of players — and that's why we needed an "env" parameters for the Player constructor
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # the random seed
        self._seed = seed
        random.seed(seed)
        
        
    def play_games(self, nb_games: int = 10000):
        """
        Simulate games and update the players history.
        
        Params
        ------
            nb_games: int
                The number of games to play.
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # let's pick 4 players randomly
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill
                )
            except:
                continue

Let's recover the skill distribution we already used.

In [ ]:
skill_dist = skills_df["skill"].values
nb_dummy = len(players) - len(skill_dist)

In [ ]:
# We'll stick to the LLM20 guessed settings, but you can define several competitions to compare them
LLM20 = Competition(skill_dist, nb_dummy)

In [ ]:
# Expect ~1700 games per second on Kaggle's systems
# Note that the history is not reset, the games add to the previous ones
LLM20.play_games(100000)

# Plots

Before diving into the results of this simulation, let's keep in mind the distribution of the actual number of games played by each submission in the real world, after two months of competition :

In [ ]:
plt.figure(figsize=(12,6))
last_games_df.groupby("guesser_SubmissionId").size().hist(bins=15)
_ = plt.title("Actual distribution of games played")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("Number of bots")

In [ ]:
last_games_df["guesser_SubmissionId"].nunique(), (last_games_df.groupby("guesser_SubmissionId").size()>100).sum()

Some bots played much more games as others because they're much older.  Actually, about half bots played less than 100 games, while some others played more than 500 !  This is an important point that we will just ignore further on, but it's good to have it in mind when comparing to the actual competition.

Now let's make some plots of our simulation to see how this ranking system performs in the long run.

In [ ]:
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('viridis')
for i in range(len(LLM20.players))[::-1]:  # players sorted by increasing skill, to get the best on top
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(pskill) if pskill>0 else [.6]*4  # bad bots in grey
    )
_ = plt.title("Mu evolution")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")

The above plot shows that although there is *a trend*, the system fails to distinguish players according to their skill, even after a large number of games.  The gray lines correspond to dummy bots.

Let's have a closer look at the behaviour of the top 10 simulated players :

In [ ]:
NFIRSTS = 10
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
for i in range(NFIRSTS):
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(i),
        label = f"skill = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("Mu evolution of top 10 (simulated) players")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")

The best players are struggling to keep up with poorer ones…

It may be interesting to compare the evolution of players with the same skill.  We could set a specific competition environment to test that, but I'm more interesting in keeping the most realistic scenario, so let's just check among our current pool of players :

In [ ]:
from collections import Counter

In [ ]:
duplicated_skills = Counter(skill_dist[:15]).most_common(2)
duplicated_skills

We have two couples of interesting candidates among the 15 best players.  Let's check how they performed over time.

In [ ]:
SHOW_ALL = True
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history],
            c = gray
        )
for i in range(20):
    pskill = LLM20.players[i].skill
    if pskill == duplicated_skills[0][0]:
        color = cmap(0)
    elif pskill == duplicated_skills[1][0]:
        color = cmap(1)
    else:
        continue
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = color,
        label = f"skill = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("Mu evolution of players with identical skills")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")

Although there are some less/more skilled competitors inbetween, hat's not that bad, considering that @c-number, who owns currently the first place of the actual leaderboard, reported that their three identical submissions [were spread on 60 positions](https://www.kaggle.com/competitions/llm-20-questions/discussion/520928), with a score varying from $\mu=1143$ to… $\mu=767$ !!!

# Comparing with the leaderboard

Let's check how our simulated leaderboard compares with the actual one :

In [ ]:
lb = pd.read_csv("/kaggle/input/llm-20-lb-2024-07-14/llm-20-questions-publicleaderboard-2024-07-14.csv")

In [ ]:
plt.figure(figsize=(12,6))
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "Leaderboard"
)
_ = plt.hist(
    [ p.history[-1].mu for p in LLM20.players.values() ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "Simulation"
)
_ = plt.title("Played games corrected leaderboard distribution")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("Number of bots")
_ = plt.legend()

That's far from perfect, but not too bad either.  Part of the differences can be explained by the different number of games played, but it's not enough, see the corrected comparison :

In [ ]:
plt.figure(figsize=(12,6))
played_dist = last_games_df.groupby("guesser_SubmissionId").size().values
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "Leaderboard"
)
_ = plt.hist(
    [
        p.history[min(len(p.history)-1, random.choice(played_dist))].mu  # we pick the "last game" number from the played games distribution
        for p in LLM20.players.values()
    ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "Simulation"
)
_ = plt.title("Played games corrected leaderboard distribution")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("Number of bots")
_ =  plt.legend()

Another point is the skill distribution being shifted towards higher values, but again it doesn't suffice to explain the difference, so I think maybe the value we set for $\beta$ or $\tau$ may be off somehow.  There is also the possibility that the parameters were adjusted along the competition.

Anyway, from my tests I strongly doubt it would change the outcome of this simulation.

# Comparing with the top10 games history

I manually scrapped the game history of the leaderboard's top 10 positions and processed the data into a dictionary which can be found as a pickle in the [LLM-20-top10-LB-matches](https://www.kaggle.com/datasets/gguillard/llm-20-top10-lb-matches) dataset.

In [ ]:
import pickle
with open("/kaggle/input/llm-20-top10-lb-matches/matches.pickle", "rb") as f:
    top10_games_dict = pickle.load(f)

How does their evolution compares with our simulation ?

In [ ]:
plt.figure(figsize=(12,6))
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history[:200]],
            c = gray
        )
for kaggler, df in top10_games_dict.items():
    _ = plt.plot(df[kaggler], label = kaggler)
_ = plt.title("Top 10 leaderboard mu evolution (background = simulation)")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")
_ = plt.legend()

The point of this plot is just to show that the leaderboard behaviour is on par with our simulation.  Note the impressive remontada from SpiralTip…

# Team matching

There is an important point that we didn't take into account in all this process.  Quoting the competition overview : 

> Each submission will play episodes (games) against other bots on the leaderboard that have a similar skill rating.

AFAICS, the TrueSkill library doesn't provide a convenient function to draw players according to their skill rating.  Is it worth bothering about this point ?  Since we have the top 10 players games history, we can check how they were paired along their $\mu$ evolution :

In [ ]:
plt.figure(figsize=(12,6))
for kaggler, df in top10_games_dict.items():
    _ = plt.scatter(df[kaggler], df["Op1"], s = 1, label = kaggler)
    _ = plt.scatter(df[kaggler], df["Op2"], s = 1)
    _ = plt.scatter(df[kaggler], df["Op3"], s = 1)
_ = plt.legend()

Although there is some trend, it is not obvious how to infer a simulation function from that.  Furthermore, since the high scoring bots can still be paired with dummy bots, their ratings can still be degraded (although it's somehow mitigated by their smaller $\sigma$).

But most of all, this doesn't address (and on the contrary enforce) the "[pit of dumbness](https://www.kaggle.com/competitions/llm-20-questions/discussion/514628#2889458)" issue, as @loh-maa named it very acurately.

# Conclusion

Many things are probably missing in this simulation.  In particular, the fact that there are new submissions over time, the fact that the game frequency decreases over time for each player (maybe in relation to sigma ?), or the fact that to a certain extent, players are matched against players of a similar rank.  I don't think any of these "feature" would change my conclusions, though.  It is clear to me that in its current state, the ranking system of the LLM 20 Questions competition gives more room to chance than to performance.

# Playground

If you wanna just simulate a TrueSkill environment without bothering (re)reading everything above, here is everything you need.

In [ ]:
!pip install trueskill

In [ ]:
import random
from typing import List, Optional, Tuple
import warnings

from tqdm import tqdm
from trueskill import TrueSkill

In [ ]:
class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # ignore this for now, this will be useful later
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # start with environment default values, i.e. (600,300)
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill],
    debug: bool = False
):
    """
    Plays a match between two teams.
    
    Params
    ------
        teamA: Tuple[Player, Player]
            A team of two players.
        teamB: Tuple[Player, Player]
            Another team of two players.
        debug: boolean
            A flag for debugging.
    """
    if debug:
        print("Players skills :", [p.skill for p in teamA+teamB])
        print("Players ratings :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # for each team, if any player is a dummy bot, the team fails to find the keyword
    # else the team finds the keyword if its guesser's skill is higher than a random number
    # (beware of the reversed logic in the code)
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # new ratings are the results of the game
    new_ratings = env.rate(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # update all players' ratings
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("Game outcome :", ranks)
        print("New ratings :", new_ratings)
    
    return

In [ ]:
class Competition:
    """
    A configurable TrueSkill competition environment.
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42,
        debug: bool = False
    ):
        """
        Constructor.
        
        Params
        ------
            skill_dist: List[float]
                The probability to find the keyword, for each "good" submission.
            nb_dummy: int
                The number of dummy bots
            mu: float
                The TrueSkill mu parameter.
            sigma: float
                The TrueSkill sigma parameter.
            beta: float
                The TrueSkill beta parameter.
            tau: float
                The TrueSkill tau parameter.
            draw_probability: float
                The TrueSkill draw_probability parameter.
            seed: float
                A random seed.
            debug: bool
                A debugging flag.
        """
        # our competition environment
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # our pool of players — and that's why we needed an "env" parameters for the Player constructor
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # the random seed
        self._seed = seed
        random.seed(seed)
        
        self.debug = debug
        
        
    def play_games(self, nb_games: int = 10000):
        """
        Simulate games and update the players history.
        
        Params
        ------
            nb_games: int
                The number of games to play.
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # let's pick 4 players randomly
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill,
                    debug = self.debug
                )
            except Exception as e:
                warnings.warn(f"Caught exception {e}")
                continue

In [ ]:
nb_players = 10  # number of actual players
nb_dummy_players = 10  # number of dummy ("bad") bots
skill_distribution = [ random.random() for i in range(nb_players)]  # skill distribution

In [ ]:
# you can define several competitions to compare them
C1 = Competition(skill_distribution, nb_dummy_players)
C2 = Competition(skill_distribution, nb_dummy_players, mu = 500, sigma = 100, beta = 200, tau = 10, draw_probability = .8, seed = 123)

In [ ]:
# Expect ~1700 games per second on Kaggle's systems
# Note that the history is not reset, the games add to the previous ones
C1.play_games(10000)
C2.play_games(10000)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot([ r.mu for r in C1.players[0].history])

# Variability over seed

In [ ]:
C = [Competition(skill_dist, nb_dummy, seed=i) for i in range(4)]

In [ ]:
for comp in C:
    comp.play_games(50000)

In [ ]:
def plot_first_n(comp, nfirsts, a):
    #fig = plt.figure(figsize=(16,8))
    cmap = plt.get_cmap('tab10')
    for i in range(nfirsts):
        pskill = comp.players[i].skill
        a.plot(
            [r.mu for r in comp.players[i].history],
            c = cmap(i),
            label = f"skill = {pskill:.2f}"
        )
    _ = a.legend()
    _ = a.set_title("Mu evolution of top 10 (simulated) players")
    _ = a.set_xlabel("Number of games played")
    _ = a.set_ylabel("TrueSkill's mu value")

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(16,16), constrained_layout=True)
for i,comp in enumerate(C):
    #plt.axes(ax[i])
    plot_first_n(comp, 5, ax[i])

In [ ]:
for comp in C:
    scores = {p: v.rating.mu for p,v in comp.players.items()}
    leaderboard = {k:v for k,v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}
    print("Leaderboard")
    print("rank\tplayer\tskill\tmu")
    for i in range(5):
        pnum = list(leaderboard.keys())[i]
        print(f"{i+1}\t{pnum}\t{comp.players[pnum].skill:.2f}\t{leaderboard[i]:.0f}")
    print("-------------")